In [1]:
from fractalai.model import RandomDiscreteModel
from fractalai.environment import ExternalProcess, ParallelEnvironment, AtariEnvironment
from fractalai.swarm_wave import SwarmWave
import time

In [2]:
# SwarmWave?

# This algorithhm is made for solving discrete Markov decision processes when a perfect model is available

In this context, if you need to generate data to train a neural network, IO is now your bottleneck.

In [3]:
name = "MsPacman-ram-v0"
skip_frames = 0  # Number of frames to skip at the beginning
dt_mean = 5  # Apply the same action n times in average
dt_std = 2 # Repeat same action a variable number of times
n_samples = 8 * 1e6  # Maximum number of samples allowed
reward_limit = 10000  # Stop the sampling when this score is reached
n_walkers = 100  # Maximum witdh of the tree containing al the trajectories
render_every = 10  # print statistics every n iterations.
balance = 1  # Balance exploration vs exploitation
save_data = True # Save the data generated


In [4]:
env = ParallelEnvironment(name=name,env_class=AtariEnvironment,
                          blocking=False, n_workers=8)  # We will play an Atari game
model = RandomDiscreteModel(max_wakers=n_walkers,
                            n_actions=env.n_actions) # The Agent will take discrete actions at random

In [ ]:
s = SwarmWave(model=model, env=env, n_walkers=n_walkers, reward_limit=reward_limit, 
              render_every=render_every, balance=balance, save_data=save_data,
              dt_mean=dt_mean, dt_std=dt_std, accumulate_rewards=True)
s.run_swarm(print_swarm=True)


Environment: MsPacman-ram-v0 | Walkers: 100 | Deaths: 0 | data_size 131
Total samples: 195368 Progress: 100.01%
Reward: mean 9607.61 | Dispersion: 721.00 | max 10001.00 | min 9280.00 | std 199.99
Episode length: mean 2266.06 | Dispersion 68.00 | max 2301.00 | min 2233.00 | std 14.69
Dt: mean 4.98 | Dispersion: 9.00 | max 10.00 | min 1.00 | std 1.76
Status: Score limit reached.
Efficiency 0.85%
Generated 1669 Examples | 117.06 samples per example.



In [ ]:
s.render_game()